# PyTorch Tweet Sentiment Analysis


## Dataset

I will be working with a slightly curated version of the Twitter US Airline Sentiment dataset available on Kaggle. It includes the text of airline-related tweets categorized as positive, negative, and neutral.


## Preparation

We need the newest version of `torchtext`, so I need to upgrade whatever Colab has to the latest version:

```
!pip install --upgrade torch==1.7.1 torchtext==0.8.1 torchvision==0.8.2
```

I also need to [install spaCy](https://spacy.io/usage) inside the notebook, using `pip` as above. I will only be using its (excellent) tokenizer and the `en_core_web_sm` pipeline.

I created a `Dataset` that reads the tweets and classes from the dataset, creates a vocabulary, and returns the sentiment labels and the tweet texts as PyTorch tensors.

- The dataset is a CSV file. Pandas is a fire-and-forget way to manipulate CSVs.

- The sentiment labels are texts, which does not directly work with cross-entropy loss. I will need to convert the labels to the numbers 0, 1, and 2.

After creating the class, I ran tests to see that it works correctly: I can retrieve some samples and the corresponding classes as tensors, and convert those back to text.

In [ ]:
!pip install --upgrade torch==1.7.1 torchtext==0.8.1 torchvision==0.8.2
!pip install tqdm==4.50.0

Requirement already up-to-date: torch==1.7.1 in /usr/local/lib/python3.7/dist-packages (1.7.1+cu101)
Requirement already up-to-date: torchtext==0.8.1 in /usr/local/lib/python3.7/dist-packages (0.8.1)
Requirement already up-to-date: torchvision==0.8.2 in /usr/local/lib/python3.7/dist-packages (0.8.2+cu101)


In [ ]:
import pandas as pd
import torch, torchtext
from torch import nn, optim, functional as F
from tqdm.auto import tqdm

In [ ]:
ds_test = TweetDataset("/content/drive/MyDrive/airline-tweets.csv.xz")

1lines [00:00, 20.00lines/s]


In [ ]:
# Test to extract entries from the full dataset
xs = ds_test[2]
print(ds_test[2])  # print out tuple of (sentiment data, vocab tensor)
print("tensor output: " + ' '.join([ds_test.vocab.itos[x] for x in xs[1]]) + "\n") 

xs = ds_test[3]
print(ds_test[3]) # print out tuple of (sentiment data, vocab tensor)
print("tensor output: " + ' '.join([ds_test.vocab.itos[x] for x in xs[1]]))

(1, tensor([  90,    4,   92,   26,  115,   53, 2390,  614,    4,   99,    3,  169,
         188,  213,    6]))
tensor output: @VirginAmerica I did n't today ... Must mean I need to take another trip !

(0, tensor([  90,   25,   34,  194, 4788,    3, 6743, 7357,   51, 1170,   51,   21,
          32, 4010,  317, 4983,   63,   69,   58,   71,   29,  536, 3504]))
tensor output: @VirginAmerica it 's really aggressive to blast obnoxious " entertainment " in your guests ' faces & amp ; they have little recourse


In [ ]:
ds_full = TweetDataset("/content/drive/MyDrive/airline-tweets.csv.xz")

n_all = len(ds_full)
n_train = int(0.8 * n_all)
n_test = n_all - n_train
rng = torch.Generator().manual_seed(69)
ds_train, ds_test = torch.utils.data.random_split(ds_full, [n_train, n_test], rng)

1lines [00:00, 21.39lines/s]


In [ ]:
# Test to get entries out from the test and training datasets
a = ds_train[0]
print(a)
print("tensor output: " + ' '.join([ds_full.vocab.itos[x] for x in a[1]]))

b = ds_test[0]
print(b)
print("tensor output: " + ' '.join([ds_full.vocab.itos[x] for x in b[1]]))

(2, tensor([  12,  340,  483,   56,  179,    2,  221, 7698,  318,   21,  465,    2,
        1260, 1481, 2083,    6]))
tensor output: @united So far so good . Just stepped down in Denver . Next Stop Portland !
(1, tensor([ 90,   4,  57,  92,   9, 107,  50,   4, 193,   7, 298, 170,  29,   3,
        420, 476,  28,   7]))
tensor output: @VirginAmerica I just did , how can I DM ? Do u have to also add me ?


Created an embedding-bag text classifier that works with the new dataset. It uses both the embedding bag, and the offset-based batching.

I've shown the training log (epochs, loss, accuracies), and shown the best validation accuracy I could able achieve.

I extracted two samples from the validation dataset: one that was classified correctly, and one that was classified incorrectly. The model was run on each of the two samples.

In [ ]:
class TextCats(nn.Module):
    def __init__(self, n_words, emb_dim, n_cats):
        super().__init__()
        self.embedding = nn.EmbeddingBag(n_words, emb_dim, mode='mean', sparse=True)
        self.fc = nn.Linear(emb_dim, n_cats)
        nn.init.xavier_uniform_(self.embedding.weight.data)
        nn.init.xavier_uniform_(self.fc.weight.data)
    def forward(self, text, offs):
        emb = self.embedding(text, offs)
        return self.fc(emb)

In [ ]:
model = TextCats(len(ds_full.vocab), 32, 3)
device = torch.device('cpu')
model.to(device)

TextCats(
  (embedding): EmbeddingBag(19137, 32, mode=mean)
  (fc): Linear(in_features=32, out_features=3, bias=True)
)

In [ ]:
def mk_offsets(samples):
    txts = [txt for _, txt in samples]
    lens = [0] + [len(txt) for txt in txts] # Computes lengths of each text sample
    offs = torch.tensor(lens[:-1]).cumsum(0) # lens[:-1] drops last element
    labs = torch.tensor([lab for lab, txt in samples]) # lab, txt tuples in samples
    txts = torch.cat(txts)
    return labs, txts, offs

In [ ]:
device = torch.device('cpu')

def run_test(model, ds, crit):
    model.eval()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=16, collate_fn=mk_offsets)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='test iter'):
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        with torch.no_grad():
            outs = model(txts, offs)
            loss = crit(outs, labs)
            total_loss += loss.item()
            total_acc += (outs.argmax(1) == labs).sum().item()
    return total_loss / len(ds), total_acc / len(ds)

def run_train(model, ds, crit, opt, sched):
    model.train()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=16, collate_fn=mk_offsets, shuffle=True)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='train iter'):
        opt.zero_grad()
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        outs = model(txts, offs)
        loss = crit(outs, labs)
        loss.backward()
        opt.step()
        total_loss += loss.item()
        total_acc += (outs.argmax(1) == labs).sum().item()
    sched.step()
    return total_loss / len(ds), total_acc / len(ds)

def run_all(model, test_ds, train_ds, crit, opt, sched, n_epochs=10):
    for epoch in tqdm(range(n_epochs), desc='epochs'):
        train_loss, train_acc = run_train(model, train_ds, crit, opt, sched)
        test_loss, test_acc = run_test(model, test_ds, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f} acc {train_acc:.4f}   test loss {test_loss:.6f} acc {test_acc:.4f}')        

In [ ]:
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.75)

run_all(model, ds_test, ds_train, crit, opt, sched, 10)

epoch 0   train loss 0.046534 acc 0.6884   test loss 0.038701 acc 0.7480


epoch 1   train loss 0.034963 acc 0.7747   test loss 0.034604 acc 0.7780


epoch 2   train loss 0.030594 acc 0.8117   test loss 0.033000 acc 0.7893


epoch 3   train loss 0.027937 acc 0.8321   test loss 0.032437 acc 0.8019


epoch 4   train loss 0.026081 acc 0.8456   test loss 0.032619 acc 0.8012


epoch 5   train loss 0.024723 acc 0.8572   test loss 0.032022 acc 0.8019


epoch 6   train loss 0.023711 acc 0.8624   test loss 0.031867 acc 0.7988


epoch 7   train loss 0.022997 acc 0.8693   test loss 0.031365 acc 0.8070


epoch 8   train loss 0.022437 acc 0.8737   test loss 0.031305 acc 0.8067


epoch 9   train loss 0.022011 acc 0.8749   test loss 0.031526 acc 0.8026



In [ ]:
#function to get a single incorrect and correct result given a trained model

device = torch.device('cpu')

def run_test_single(model, ds, crit):
    correct_sample = None
    incorrect_sample = None

    flag_incorr = False
    flag_corr = False

    model.eval()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=1, collate_fn=mk_offsets)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='test iter'):
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        with torch.no_grad():
            outs = model(txts, offs)
            loss = crit(outs, labs)
            total_loss += loss.item()
            total_acc += (outs.argmax(1) == labs).sum().item()
            if not flag_corr and (outs.argmax(1) == labs):
              correct_sample = (labs[0], txts)
              correct_sentiment = labs[0]
              flag_corr = True
            if not flag_incorr and (outs.argmax(1) != labs):
              incorrect_sample = (labs[0], txts)
              incorrect_sentiment = outs.argmax(1)
              actual_sentiment = labs[0]
              flag_incorr = True
            if flag_incorr and flag_corr:
              return correct_sample, incorrect_sample, correct_sentiment, incorrect_sentiment, actual_sentiment

def get_corr_incorr(model, test_ds, train_ds, crit, opt, sched):
    
    correct_sample, incorrect_sample, correct_sentiment, incorrect_sentiment, actual_sentiment = run_test_single(model, test_ds, crit)

    print(correct_sample[1])
    print("Correct Sample: " + ' '.join([ds_full.vocab.itos[x] for x in correct_sample[1]]))
    print("predicted correct sentiment: "+ str(correct_sentiment))
    print(incorrect_sample[1])
    print("Incorrect Sample: " + ' '.join([ds_full.vocab.itos[x] for x in incorrect_sample[1]]))
    print("predicted sentiment: " + str(incorrect_sentiment) + ". Actual sentiment: " + str(actual_sentiment))

In [ ]:
# This uses the same model as the one before.
get_corr_incorr(model, ds_test, ds_train, crit, opt, sched)

tensor([ 90,   4,  57,  92,   9, 107,  50,   4, 193,   7, 298, 170,  29,   3,
        420, 476,  28,   7])
Correct Sample: @VirginAmerica I just did , how can I DM ? Do u have to also add me ?
predicted correct sentiment: tensor(1)
tensor([ 22,  17,  16,  36, 696,  20,  88,  15,   4,  66, 274,  17,  16,  21,
        465,   9,  20,  75,   8, 131,   3, 275,  62,  86,  71,  66,  70,   5,
         74,   7])
Incorrect Sample: @SouthwestAir my flight at BWI is delayed and I will miss my flight in Denver , is there a way to find out if they will hold the plane ?
predicted sentiment: tensor([0]). Actual sentiment: tensor(1)


Results from training/testing on full model:

* epoch 0   train loss 0.046534 acc 0.6884   test loss 0.038701 acc 0.7480
*epoch 1   train loss 0.034963 acc 0.7747   test loss 0.034604 acc 0.7780
*epoch 2   train loss 0.030594 acc 0.8117   test loss 0.033000 acc 0.7893
*epoch 3   train loss 0.027937 acc 0.8321   test loss 0.032437 acc 0.8019
*epoch 4   train loss 0.026081 acc 0.8456   test loss 0.032619 acc 0.8012
*epoch 5   train loss 0.024723 acc 0.8572   test loss 0.032022 acc 0.8019
*epoch 6   train loss 0.023711 acc 0.8624   test loss 0.031867 acc 0.7988
*epoch 7   train loss 0.022997 acc 0.8693   test loss 0.031365 acc 0.8070
*epoch 8   train loss 0.022437 acc 0.8737   test loss 0.031305 acc 0.8067
*epoch 9   train loss 0.022011 acc 0.8749   test loss 0.031526 acc 0.8026

A Incorrect and Correct Result:

* Correct Sample: tensor([ 90,   4,  57,  92,   9, 107,  50,   4, 193,   7, 298, 170,  29,   3, 420, 476,  28,   7])
* Correct Sample Text: @VirginAmerica I just did , how can I DM ? Do u have to also add me ?
  * predicted correct sentiment: tensor(1)
Incorrect:

* Incorrect Sample: tensor([ 22,  17,  16,  36, 696,  20,  88,  15,   4,  66, 274,  17,  16,  21, 465,   9,  20,  75,   8, 131,   3, 275,  62,  86,  71,  66,  70,   5, 74,   7])
* Incorrect Sample Text: @SouthwestAir my flight at BWI is delayed and I will miss my flight in Denver , is there a way to find out if they will hold the plane ?
* Incorrectly predicted sentiment: tensor([0]). Actual sentiment: tensor(1)

**Bonus**: Another incorrect sample from a lesser-trained model
* Incorrect Sample: tensor([  12,  114,  144,  143,    3, 2788,  374,   24,   32,  628,  389,    2, 4478,   10,  352,  174, 3424,  698,   36,   97,    2, 4455,    2])
* Incorrect Sample: @united You 're trying to solve problem of your own making . Charging for checked luggage forces checking at gate . Brilliant .
* predicted sentiment: tensor([1]). Actual sentiment: tensor(0)

Pretty interesting! I don't think the model understands sarcasm at all here. When reading the sentence normally, nothing immediately indicates positive or negative tone. But if I imagine someone saying this, I already know it's sarcastic and negative.


## Applying ngrams

The model will be expanded to consider any sequential information in the samples instead of treating each word as independent.

I will embed _n-grams_ into the mdoel. An n-gram is a short sequence of up to _n_ words that appears in the text: for example, if _n_=2, the text "Mary had a little lamb" will yield the n-grams "Mary", "had", "a", "little", "lamb", "Mary had", "had a", "a little", and "little lamb".

The easiest way to do this is to apply `torchtext.data.utils.ngrams_iterator()` to the token sequence. The new model is trained and samples are generated using n-grams applied.

In [ ]:
device = torch.device('cpu')

# I just added ngrams parameter to the TweetDataset to the token sequence. These are the the functions above needed for Task 3

class TweetDataset2(torch.utils.data.Dataset):
  
  def __init__(self, xz_compressed_file, n_grams, vocab=None):

    self.tokenizer = torchtext.data.utils.get_tokenizer('spacy', 'en_core_web_sm')
    self.data = pd.read_csv(xz_compressed_file)
    self.n_grams = n_grams

    # Form vocabulary. Update toks to have n_grams applied to it
    text_data = ' '.join(self.data["text"])
    toks = self.tokenizer(text_data) 
    toks = torchtext.data.utils.ngrams_iterator(toks, self.n_grams)

    if vocab is None:
      self.vocab = torchtext.vocab.build_vocab_from_iterator([toks])
    else:
      self.vocab = vocab
    self.toks = torch.LongTensor([self.vocab[tok] for tok in toks])


    # Convert sentiment data to integers
    self.sentiment_data = self.data["sentiment"]
    for i in range(0, len(self.sentiment_data)):
      if self.sentiment_data[i] == 'neutral':
        self.sentiment_data[i] = 1
      elif self.sentiment_data[i] == 'positive':
        self.sentiment_data[i] = 2
      elif self.sentiment_data[i] == 'negative':
        self.sentiment_data[i] = 0
  
  def __len__(self):
      return len(self.data["sentiment"])

  def __getitem__(self, i):
      # print(torch.LongTensor([self.vocab[tok] for tok in list(torchtext.data.utils.ngrams_iterator(self.tokenizer(self.data["text"][i]), self.n_grams))]).shape)
      return (self.sentiment_data[i], torch.LongTensor([self.vocab[tok] for tok in list(torchtext.data.utils.ngrams_iterator(self.tokenizer(self.data["text"][i]), self.n_grams))]))

def run_test_t4(model, ds, crit):
    model.eval()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=16, collate_fn=mk_offsets_t4)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='test iter'):
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        with torch.no_grad():
            outs = model(txts, offs)
            loss = crit(outs, labs)
            total_loss += loss.item()
            total_acc += (outs.argmax(1) == labs).sum().item()
    return total_loss / len(ds), total_acc / len(ds)

class TextCats_t4(nn.Module):
    def __init__(self, n_words, emb_dim, n_cats):
        super().__init__()
        self.embedding = nn.EmbeddingBag(n_words, emb_dim, mode='mean', sparse=True)
        self.fc = nn.Linear(emb_dim, n_cats)
        nn.init.xavier_uniform_(self.embedding.weight.data)
        nn.init.xavier_uniform_(self.fc.weight.data)
    def forward(self, text, offs):
        emb = self.embedding(text, offs)
        return self.fc(emb)

def run_train_t4(model, ds, crit, opt, sched):
    model.train()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=16, collate_fn=mk_offsets_t4, shuffle=True)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='train iter'):
        opt.zero_grad()
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        outs = model(txts, offs)
        loss = crit(outs, labs)
        loss.backward()
        opt.step()
        total_loss += loss.item()
        total_acc += (outs.argmax(1) == labs).sum().item()
    sched.step()
    return total_loss / len(ds), total_acc / len(ds)

def run_all_t4(model, test_ds, train_ds, crit, opt, sched, n_epochs=10):
    for epoch in tqdm(range(n_epochs), desc='epochs'):
        train_loss, train_acc = run_train_t4(model, train_ds, crit, opt, sched)
        test_loss, test_acc = run_test_t4(model, test_ds, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f} acc {train_acc:.4f}   test loss {test_loss:.6f} acc {test_acc:.4f}')        

def mk_offsets_t4(samples):
    txts = [txt for _, txt in samples]
    lens = [0] + [len(txt) for txt in txts] # Computes lengths of each text sample
    offs = torch.tensor(lens[:-1]).cumsum(0) # lens[:-1] drops last element
    labs = torch.tensor([lab for lab, txt in samples]) # lab, txt tuples in samples
    txts = torch.cat(txts)
    return labs, txts, offs

#function to get a single incorrect and correct result

def run_test_single_t4(model, ds, crit):
    correct_sample = None
    incorrect_sample = None

    flag_incorr = False
    flag_corr = False

    model.eval()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=1, collate_fn=mk_offsets_t4)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='test iter'):
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        with torch.no_grad():
            outs = model(txts, offs)
            loss = crit(outs, labs)
            total_loss += loss.item()
            total_acc += (outs.argmax(1) == labs).sum().item()
            if not flag_corr and (outs.argmax(1) == labs):
              correct_sample = (labs[0], txts)
              correct_sentiment = labs[0]
              flag_corr = True
            if not flag_incorr and (outs.argmax(1) != labs):
              incorrect_sample = (labs[0], txts)
              incorrect_sentiment = outs.argmax(1)
              actual_sentiment = labs[0]
              flag_incorr = True
            if flag_incorr and flag_corr:
              return correct_sample, incorrect_sample, correct_sentiment, incorrect_sentiment, actual_sentiment

def get_corr_incorr_t4(model, test_ds, train_ds, crit, opt, sched):
    
    correct_sample, incorrect_sample, correct_sentiment, incorrect_sentiment, actual_sentiment = run_test_single_t4(model, test_ds, crit)

    print(correct_sample[1])
    print("Correct Sample: " + ' '.join([ds_t4.vocab.itos[x] for x in correct_sample[1]]))
    print("predicted correct sentiment: "+ str(correct_sentiment))
    print(incorrect_sample[1])
    print("Incorrect Sample: " + ' '.join([ds_t4.vocab.itos[x] for x in incorrect_sample[1]]))
    print("predicted sentiment: " + str(incorrect_sentiment) + ". Actual sentiment: " + str(actual_sentiment))

In [ ]:
n_grams = 6

ds_t4 = TweetDataset2("/content/drive/MyDrive/airline-tweets.csv.xz", n_grams=n_grams)

model = TextCats_t4(len(ds_t4.vocab), 32, 3)
device = torch.device('cpu')
model.to(device)

crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.75)

n_all = len(ds_t4)
n_train = int(0.8 * n_all)
n_test = n_all - n_train
rng = torch.Generator().manual_seed(69)
ds_train, ds_test = torch.utils.data.random_split(ds_t4, [n_train, n_test], rng)

# Test to get entries out from the test and training datasets with ngrams
a = ds_train[0]
print(a)
print("tensor output: " + ' '.join([ds_t4.vocab.itos[x] for x in a[1]]))

b = ds_test[0]
print(b)
print("tensor output: " + ' '.join([ds_t4.vocab.itos[x] for x in b[1]]))

run_all_t4(model, ds_test, ds_train, crit, opt, sched, 10)

get_corr_incorr_t4(model, ds_train, ds_test, crit, opt, sched)

1lines [00:01,  1.72s/lines]


(2, tensor([     12,     539,     812,      60,     234,       2,     300,   84461,
            486,      21,     772,       2,    3258,    4150,    7181,       6,
           7090,   10737,   22130,    9907,    2483,     665,  418888, 1048363,
           9441,    2991,    5813,    4794,  433785,  457118,  445229,  330090,
          57578,   22131,   37244,  744459,  205350,  418889, 1048364,  663859,
          22863,  369195,  207104,  433786,  457119,  330091,   57579,   67964,
        1038178,  744460,  205351,  418890, 1048365,  663860,  800051,  369196,
         207105,  433787,  330092,  455025,  682807, 1038179,  744461,  205352,
         418891, 1048366,  663861,  800052,  369197,  207106,  330093,  455026,
         682808, 1038180,  744462,  205353,  418892, 1048367,  663862,  800053,
         369198]))
tensor output: @united So far so good . Just stepped down in Denver . Next Stop Portland ! @united So So far far so so good good . . Just Just stepped stepped down down in in De

KeyboardInterrupt: ignored

n = 1 
* epoch 0   train loss 0.046268 acc 0.6924   test loss 0.038151 acc 0.7565
*epoch 1   train loss 0.034815 acc 0.7764   test loss 0.034542 acc 0.7801
*epoch 2   train loss 0.030438 acc 0.8140   test loss 0.033354 acc 0.7848
*epoch 3   train loss 0.027893 acc 0.8332   test loss 0.032407 acc 0.7975
*epoch 4   train loss 0.026005 acc 0.8450   test loss 0.032857 acc 0.7982
*epoch 5   train loss 0.024687 acc 0.8564   test loss 0.031670 acc 0.8040
*epoch 6   train loss 0.023702 acc 0.8645   test loss 0.031565 acc 0.8043
*epoch 7   train loss 0.022930 acc 0.8700   test loss 0.031366 acc 0.8050
*epoch 8   train loss 0.022407 acc 0.8723   test loss 0.031384 acc 0.8050
*epoch 9   train loss 0.022009 acc 0.8758   test loss 0.031453 acc 0.8053

n = 2
* epoch 0   train loss 0.051476 acc 0.6562   test loss 0.046765 acc 0.7104
*epoch 1   train loss 0.038755 acc 0.7454   test loss 0.037202 acc 0.7678
*epoch 2   train loss 0.032332 acc 0.7976   test loss 0.036108 acc 0.7753
*epoch 3   train loss 0.028278 acc 0.8299   test loss 0.033844 acc **0.7893**
*epoch 4   train loss 0.025482 acc 0.8526   test loss 0.033268 acc 0.7920
*epoch 5   train loss 0.023479 acc 0.8665   test loss 0.033048 acc 0.7923
*epoch 6   train loss 0.022084 acc 0.8792   test loss 0.032776 acc 0.7964
*epoch 7   train loss 0.021020 acc 0.8859   test loss 0.032502 acc 0.8016
*epoch 8   train loss 0.020263 acc 0.8916   test loss 0.032408 acc 0.7985
*epoch 9   train loss 0.019714 acc 0.8957   test loss 0.032367 acc 0.8009


n = 3

* epoch 0   train loss 0.054376 acc 0.6391   test loss 0.049059 acc 0.6633
*epoch 1   train loss 0.043928 acc 0.7018   test loss 0.041311 acc 0.7312
*epoch 2   train loss 0.036987 acc 0.7591   test loss 0.038462 acc 0.7534
*epoch 3   train loss 0.032443 acc 0.7978   test loss 0.036897 acc 0.7766
*epoch 4   train loss 0.029293 acc 0.8222   test loss 0.035546 acc 0.7801
*epoch 5   train loss 0.027066 acc 0.8428   test loss 0.035095 acc 0.7814
*epoch 6   train loss 0.025434 acc 0.8558   test loss 0.034697 acc 0.7879
*epoch 7   train loss 0.024256 acc 0.8651   test loss 0.034450 acc 0.7869
*epoch 8   train loss 0.023379 acc 0.8719   test loss 0.034388 acc 0.7879
*epoch 9   train loss 0.022745 acc 0.8774   test loss 0.034220 acc 0.7879


n = 4
* epoch 0   train loss 0.055523 acc 0.6335   test loss 0.051216 acc 0.6561
*epoch 1   train loss 0.047236 acc 0.6790   test loss 0.044871 acc 0.6977
*epoch 2   train loss 0.040620 acc 0.7302   test loss 0.041437 acc 0.7415
*epoch 3   train loss 0.036110 acc 0.7644   test loss 0.039336 acc 0.7510
*epoch 4   train loss 0.032886 acc 0.7905   test loss 0.038203 acc 0.7596
*epoch 5   train loss 0.030522 acc 0.8115   test loss 0.037366 acc 0.7664
*epoch 6   train loss 0.028803 acc 0.8260   test loss 0.036992 acc 0.7736
*epoch 7   train loss 0.027568 acc 0.8372   test loss 0.036656 acc 0.7725
*epoch 8   train loss 0.026646 acc 0.8462   test loss 0.036417 acc 0.7753
*epoch 9   train loss 0.025955 acc 0.8532   test loss 0.036257 acc 0.7756

n = 5
* epoch 0   train loss 0.055968 acc 0.6316   test loss 0.052686 acc 0.6482
*epoch 1   train loss 0.049745 acc 0.6636   test loss 0.047427 acc 0.6841
*epoch 2   train loss 0.044024 acc 0.6989   test loss 0.045018 acc 0.7203
*epoch 3   train loss 0.039674 acc 0.7347   test loss 0.041745 acc 0.7302
*epoch 4   train loss 0.036474 acc 0.7611   test loss 0.040591 acc 0.7418
*epoch 5   train loss 0.034150 acc 0.7802   test loss 0.039628 acc 0.7616
*epoch 6   train loss 0.032405 acc 0.7956   test loss 0.039070 acc 0.7565
*epoch 7   train loss 0.031128 acc 0.8058   test loss 0.038628 acc 0.7602
*epoch 8   train loss 0.030177 acc 0.8151   test loss 0.038366 acc 0.7657
*epoch 9   train loss 0.029484 acc 0.8210   test loss 0.038150 acc 0.7650

Correct and Incorrect Samples
Correct sample:
* tensor([    12,    512,    761,     60,    232,      2,    295,  36518,    462,        21,    725,      2,   2833,   3540,   5759,      6,   5697,   8133,     14558,   7627,   2204,    634,  66535, 125293,   7291,   2625,   4766,     4032,  68807,  72405,  70440])
 * Correct Sample text: @united So far so good . Just stepped down in Denver . Next Stop Portland ! @united So So far far so so good good . . Just Just stepped stepped down down in in Denver Denver . . Next Next Stop Stop Portland Portland !
* predicted correct sentiment: tensor(2)

Incorrect Sample:
* tensor([  691,  1556,   558,   157,     5,    79,   355,    10,  1943,   900, 7, 57938, 13505, 29147,  3038, 10402,  4908,  7701, 95473, 74208, 15631])
* Incorrect Sample text: @usairways Does anyone know the hold times for USAirways reservations ? @usairways Does Does anyone anyone know know the the hold hold times times for for USAirways USAirways reservations reservations ?
predicted sentiment: 

* predicted sentiment: tensor([0]). Actual sentiment: tensor(1)